In [ ]:
from vantage6.client import Client

In [ ]:
client = Client("http://localhost", 5000, "/api")
client.authenticate("node1-admin", "password")
client.setup_encryption(None)

In [ ]:
# Find out the organizations and collaborations we can run on
import json
collaboration_list = client.collaboration.list()
collaboration_index = 0
organization_ids_ = [ ]

for organization in collaboration_list[collaboration_index]['organizations']:
    organization_ids_.append(organization['id'])
print(json.dumps(client.node.list(), indent=2))

In [ ]:
# Request the structure to be built
input_ = {
    "master": "true",
    "method":"master", 
    "args": [ ],
    "kwargs": {}
}

task = client.post_task(
    name="GetGraphStructure",
    image="jaspersnel/v6-graphstructure-rdf-py",
    collaboration_id=collaboration_list[collaboration_index]['id'],#Get the first collaboration associated with user
    input_= input_,
    organization_ids=[organization_ids_[0]]
)

print(json.dumps(task, indent=2))

In [ ]:
# Wait for results
import time
import json
resultObjRef = task.get("results")[0]
resultObj = client.result.get(resultObjRef['id'])
attempts = 1
while((resultObj["finished_at"] == None) and attempts < 10):
    print("waiting...")
    time.sleep(5)
    resultObj = client.result.get(resultObjRef['id'])
    attempts += 1
results = resultObj['result']
intersect = results['intersect']
union = results['union']
print(intersect[:10])
print(union[:10])

In [ ]:
# List (some of) the classes we've found
classes = set()
for s, p, o in union + intersect:
    classes.add(s)
    classes.add(o)

for i, cls in enumerate(classes):
    print(cls)
    if i >= 9:
        break

In [ ]:
# Show all the common elements
from pyvis.network import Network

intersect_graph = Network(notebook=True, directed=True)

for s, p, o in intersect:
    intersect_graph.add_nodes([s, o])
    intersect_graph.add_edge(s, o, title=p)

intersect_graph.show('intersect.html')

In [ ]:
# Show all elements
union_graph = Network(notebook=True, directed=True)

for s, p, o in union:
    union_graph.add_nodes([s, o])
    union_graph.add_edge(s, o, title=p)

union_graph.show('union.html')

In [ ]:
# Combine them!
graph = Network(notebook=True, directed=True)

intersect_classes = [s for s, _, _ in intersect] + [o for _, _, o in intersect]

for s, p, o in union:
    color = 'grey'
    if s in intersect_classes:
        color = 'blue'
    graph.add_node(s, color=color)

    color = 'grey'
    if o in intersect_classes:
        color = 'blue'
    graph.add_node(o, color=color)

    graph.add_edge(s, o, title=p)

graph.show('integrated.html')